In [18]:
import pandas as pd
import numpy as np

data = pd.read_csv('IFR_Extract_with_selected_columns_15-5-23.csv')

obreak_date = pd.to_datetime(data.obreak_date)
datebone = pd.to_datetime(data.datebone)
y = ( abs( datebone - obreak_date))
X = data.drop(["obreak_date","datebone"],axis=1)
selectedColumns = [ 'PatientAge', "PatientGender",'parentbreak', 'alcohol',
                'arthritis', 'diabetes',
                'oralster', 'smoke', 'obreak']



In [19]:
dropList = []
for i in data:
    if data[i].dtypes == 'O':
        dropList.append(data[i].name)
dropList.append("CompletedSurveyId")
dropList.append("PatientId")
X = data.drop(dropList,axis=1)
X.fillna(0,inplace=True)
y = pd.DataFrame({"time":y})



In [20]:
y.time

y['event'] = y.time.apply(lambda x: x.days != 0 )
structured_array = y.to_records(index=False)

swapped = pd.DataFrame({
    "event": y.event,
    "time": y.time.apply(lambda x: x.days)
})
(swapped.event).value_counts()

True     792
False      8
Name: event, dtype: int64

In [21]:
swapped.event = swapped.event.astype(bool)
swapped.event
structured_array = np.rec.array(swapped.to_records(index=False))

In [22]:
swapped

,event,time
0,True,524
1,True,2046
2,True,15455
3,True,4354
4,True,2207
...,...,...
795,True,579
796,True,5109
797,True,2125
798,True,518


In [14]:
mergedBeforeEncoding = pd.concat([X[selectedColumns],swapped],axis=1)
mergedBeforeEncoding

,PatientAge,PatientGender,parentbreak,alcohol,arthritis,diabetes,oralster,smoke,obreak,event,time
0,53,1,0,0,0.0,0.0,0,0,1,True,524
1,85,1,0,0,0.0,1.0,0,0,1,True,2046
2,90,1,0,0,1.0,0.0,0,0,1,True,15455
3,81,1,0,0,0.0,0.0,0,0,1,True,4354
4,60,1,1,0,0.0,1.0,0,0,1,True,2207
...,...,...,...,...,...,...,...,...,...,...,...
795,83,1,4,0,0.0,0.0,4,0,1,True,579
796,60,1,0,0,0.0,0.0,0,0,1,True,5109
797,76,2,0,0,0.0,0.0,0,0,1,True,2125
798,61,1,2,1,0.0,0.0,0,1,1,True,518


In [15]:
PatientAge = []
PatientGender = []
parentbreak = []
alcohol = []
arthritis = []
diabetes = []
oralster = []
smoke = []
obreak = []
event = []
time = []
for i in range(400):
    PatientAge.append(np.random.randint(40,104))
    PatientGender.append( np.random.randint(1,3))
    parentbreak.append(np.random.randint(0,5))
    alcohol.append(np.random.randint(0,5))
    arthritis.append(np.random.randint(0,2))
    diabetes.append(np.random.randint(0,2))
    oralster.append(np.random.randint(0,5))
    smoke.append(np.random.randint(0,5))
    obreak.append(1)
    event.append(False)
    time.append(0)

synthetic_data = pd.DataFrame({
    "PatientAge":PatientAge,
    "PatientGender":PatientGender,
    "parentbreak":parentbreak,
    "alcohol":alcohol,
    "arthritis":arthritis,
    "diabetes":diabetes,
    "oralster":oralster,
    "smoke":smoke,
    "obreak":obreak,
    "event":event,
    "time":time
})

synthetic_data['arthritis'] = synthetic_data['arthritis'].astype(float)
synthetic_data['diabetes'] = synthetic_data['diabetes'].astype(float)

merged_synthetic = pd.concat([mergedBeforeEncoding, synthetic_data], ignore_index=True)
merged_synthetic

,PatientAge,PatientGender,parentbreak,alcohol,arthritis,diabetes,oralster,smoke,obreak,event,time
0,53,1,0,0,0.0,0.0,0,0,1,True,524
1,85,1,0,0,0.0,1.0,0,0,1,True,2046
2,90,1,0,0,1.0,0.0,0,0,1,True,15455
3,81,1,0,0,0.0,0.0,0,0,1,True,4354
4,60,1,1,0,0.0,1.0,0,0,1,True,2207
...,...,...,...,...,...,...,...,...,...,...,...
1195,44,1,0,2,0.0,0.0,3,3,1,False,0
1196,99,1,3,1,1.0,1.0,0,3,1,False,0
1197,68,1,2,2,1.0,0.0,3,2,1,False,0
1198,61,2,2,4,0.0,1.0,0,3,1,False,0


In [16]:
cat_features = ['parentbreak', 'alcohol',
                'arthritis', 'diabetes',
                'oralster', 'smoke', 'obreak',
                # These features were determined to apply minimal impact even
                # 'respdisease', 'hbp','heartdisease',
                # 'ptunsteady', 'wasfractdue2fall', 'cholesterol',
                # 'ptfall', 'shoulder', 'wrist', 'bmdtest_10yr_caroc'
                ]
for feature in cat_features:
    if merged_synthetic is not None:
        cat_one_hot = pd.get_dummies(merged_synthetic[feature], prefix=f'{feature}', drop_first=False)
        merged_synthetic = merged_synthetic.drop(feature, axis=1)
        merged_synthetic = merged_synthetic.join(cat_one_hot)

In [23]:
merged_synthetic

,PatientAge,PatientGender,event,time,parentbreak_0,parentbreak_1,parentbreak_2,parentbreak_3,parentbreak_4,alcohol_0,...,oralster_1,oralster_2,oralster_3,oralster_4,smoke_0,smoke_1,smoke_2,smoke_3,smoke_4,obreak_1
0,53,1,True,524,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
1,85,1,True,2046,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
2,90,1,True,15455,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
3,81,1,True,4354,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
4,60,1,True,2207,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,44,1,False,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
1196,99,1,False,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
1197,68,1,False,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1198,61,2,False,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
